In [14]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql

user = "root"
password = "1234"
host = "localhost"
port = 3306
database = "university_df"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}", echo=False)


def insert_ignore(engine, table_name, df):
    if df.empty:
        print(f"[❗ SKIP] {table_name} is empty.")
        return
    df = df.where(pd.notnull(df), None)  # NaN -> None

    with engine.begin() as conn:
        cols = ', '.join(f"`{col}`" for col in df.columns)
        placeholders = ', '.join([f":{col}" for col in df.columns])
        sql = text(f"INSERT IGNORE INTO {table_name} ({cols}) VALUES ({placeholders})")

        values = df.to_dict(orient='records')
        conn.execute(sql, values)

excel_path = "연구실_8개_테이블_정리.xlsx"

df1 = pd.read_excel(excel_path, sheet_name="테이블1_기본정보")
df2 = pd.read_excel(excel_path, sheet_name="테이블2_연구원")
df3 = pd.read_excel(excel_path, sheet_name="테이블3_논문")
df4 = pd.read_excel(excel_path, sheet_name="테이블4_교수_과목")
df5 = pd.read_excel(excel_path, sheet_name="테이블5_프로젝트")
df6 = pd.read_excel(excel_path, sheet_name="테이블6_리크루트")
df7 = pd.read_excel(excel_path, sheet_name="테이블7_연구주제")
df8 = pd.read_excel(excel_path, sheet_name="테이블8_교수님")

df1.columns = ['main_url', 'category', 'title', 'url']
insert_ignore(engine, 'main_info', df1)

df2.columns = ['main_url', 'category', 'name', 'career_years', 'field', 'research_topic', 'etc']
insert_ignore(engine, 'researcher', df2)

df3.columns = ['main_url', 'category', 'paper_title', 'journal', 'author', 'year']
insert_ignore(engine, 'paper', df3)

subject_cols_all = [f'과목{i}' for i in range(1, 21)]

subject_cols = [col for col in subject_cols_all if col in df4.columns]

df4['subjects'] = df4[subject_cols].apply(
    lambda x: ', '.join([str(i) for i in x if pd.notna(i) and i != '미상']),
    axis=1
)

df4_final = df4[['메인주소', '분류', 'subjects']].rename(
    columns={'메인주소': 'main_url', '분류': 'category'}
)

insert_ignore(engine, 'professor_subject', df4_final)

df5.columns = ['main_url', 'category', 'project_name', 'project_desc']
insert_ignore(engine, 'project', df5)

df6.columns = ['main_url', 'category', 'info']
insert_ignore(engine, 'recruit', df6)

df7.columns = ['main_url', 'category', 'topic']
insert_ignore(engine, 'research_topic', df7)

df8.columns = ['main_url', 'category', 'name','email', 'university', 'phone', 'office_location', 'other_info']
insert_ignore(engine, 'professor', df8)

print("✅ 모든 테이블을 중복 없이 MySQL에 성공적으로 삽입 완료!")


✅ 모든 테이블을 중복 없이 MySQL에 성공적으로 삽입 완료!
